In [1]:
import pandas as pd
import psycopg2
import csv
import re

# Geocoding using psycopg2

In [2]:
df = pd.read_csv("ticketSearchResults-2015-02-09-20_48_16.txt", sep="~")
df

,Ticket,Call Dt/Tm,Header,County,City,Excav Company,Address,Street,Inter1,Inter2
0,100600001,2010-03-01 02:51:21-05,EMERGENCY,BERGEN,ROCKLEIGH,ORANGE AND ROCKLAND UTILITIES,9,PIERMONT,HARING FARM,ROCKLEIGH
1,100600002,2010-03-01 03:00:33-05,EMERGENCY,BERGEN,ALPINE,ORANGE AND ROCKLAND UTILITIES,441,US HWY 9W,ORBACH,NaN
2,100600003,2010-03-01 04:05:36-05,EMERGENCY,MONMOUTH,HAZLET,VERIZON,NaN,LAUREL,HELFRICH,NaN
3,100600004,2010-03-01 05:15:23-05,EMERGENCY,ESSEX,FAIRFIELD,PSE&G,11,MADISON,PLYMOUTH,NaN
4,100600005,2010-03-01 05:21:24-05,EMERGENCY,PASSAIC,HALEDON,MONTANA CONSTRUCTION,89,LEE,LEGION,HENRY
...,...,...,...,...,...,...,...,...,...,...
46898,100902145,2010-03-31 23:21:10-04,ROUTINE,GLOUCESTER,WASHINGTON,STEPHENS EXTERMINATING,1701,HAWTHORNE,ARBOUR,NaN
46899,100902146,2010-03-31 23:26:30-04,ROUTINE,GLOUCESTER,WASHINGTON,STEPHENS EXTERMINATING,401,BRIARWOOD,ARBOUR,NaN
46900,100902147,2010-03-31 23:29:39-04,ROUTINE,MIDDLESEX,PLAINSBORO,TELECABLE,101,MORGAN,ENTERPRISE,NaN
46901,100902148,2010-03-31 23:37:03-04,ROUTINE,MONMOUTH,SHREWSBURY BORO,TELECABLE,149,AVENUE OF THE CMN,BROAD,NaN


In [3]:
output_header = ["Ticket",
                 "Date",
                 "Time",
                 "Header",
                 "County",
                 "Rating",
                 "Longitude",
                 "Latitude",
                 "St_number",
                 "St_name",
                 "Neighborhood",
                 "City",
                 "State",
                 "Zip",
                 "Excav_company",
                 "Inter1",
                 "Inter2"]

In [4]:
try:
    conn = psycopg2.connect("dbname='geocoder' user='postgres' host='localhost' password='postgres'")
    print("Database connected!")
except:
    print("I am unable to connect to the database")

conn.close()

Database connected!


In [16]:
epochs = 500

try:
    conn = psycopg2.connect("dbname='geocoder' user='postgres' host='localhost' password='postgres'")
    print("Database connected!")
        
    cur = conn.cursor()

    f_in = open("ticketSearchResults-2015-02-09-20_48_16.txt", 'r')
    f_out = open("test.csv", 'w')
    f_bad = open("badlines.txt", 'w')
    f_log = open("test.log", 'w')

    write_header = csv.writer(f_out, lineterminator='\n')
    write_header.writerow(output_header)

    i = 0
    n = 0
    for line in f_in.readlines():
        if not re.search(r'Ticket', line):
            rec = line.split('~')
            address = (rec[6], # street number
                       rec[7].replace("'", "''"), # street name
                       rec[4]) # city name

            expression = """SELECT g.rating, ST_X(g.geomout), ST_Y(g.geomout), (addy).address, (addy).streetname, (addy).location, (addy).stateabbrev, (addy).zip FROM geocode('%s %s, %s, NJ', 1) AS g;""" % address

            cur.execute(expression)

            try:

                rows = [str(x) for x in cur.fetchall()[0]]

                f_out.write("%s,%s,%s,%s,%s," % (
                    rec[0], # ticket
                    rec[1].split(' ')[0], # date
                    rec[1].split(' ')[1], # time
                    rec[2], # header
                    rec[3]  # county
                ))

                f_out.write("%s,%s,%s,%s,%s,%s,%s,%s," % (
                    rows[0], # rating int
                    rows[1], # longitude float.13
                    rows[2], # latitude float.13
                    rows[3], # street number int
                    rows[4], # street name string
                    rows[5], # neighborhood string
                    rows[6], # state
                    rows[7]  # zip
                ))

                f_out.write("%s,%s,%s\n" % (
                    rec[5], # excav_comp
                    rec[8], # inter1
                    rec[9].strip(), # inter2
                ))

                print("Job index %d status: complete!" % i)
                f_log.write("Job index %d status: complete!\n" % i)

            except:
                f_bad.write("%s\n" % line)
                n += 1
                print("Job index %d status: bad input line!(Total: %d)" % (i, n))
                f_log.write("Job index %d status: bad input line!(Total: %d)\n" % (i, n))
                
            i += 1
            if i == epochs:
                break
    
    f_log.write("All jobs are complete!\nTotal input = %d\nUngeocoded rate = %.4f%" % (i + 1, n / (i + 1)))
    
    f_in.close()
    f_out.close()
    f_bad.close()
    f_log.close()
    conn.close()
    print("All jobs are complete! Database disconnected!")
    
except:
    print("I am unable to connect to the database!")

Database connected!
job 0 is complete!
job 1 is complete!
job 2 is complete!
job 3 is complete!
job 4 is complete!
job 5 is complete!
job 6 is complete!
job 7 is complete!
job 8 is complete!
job 9 is complete!
job 10 is complete!
job 11 is complete!
job 12 is complete!
job 13 is complete!
job 14 is complete!
job 15 is complete!
job 16 is complete!
job 17 is complete!
Bad input line encountered!(1)
job 19 is complete!
job 20 is complete!
Bad input line encountered!(2)
job 22 is complete!
job 23 is complete!
job 24 is complete!
job 25 is complete!
job 26 is complete!
job 27 is complete!
job 28 is complete!
job 29 is complete!
job 30 is complete!
job 31 is complete!
job 32 is complete!
job 33 is complete!
job 34 is complete!
job 35 is complete!
job 36 is complete!
job 37 is complete!
job 38 is complete!
job 39 is complete!
Bad input line encountered!(3)
job 41 is complete!
Bad input line encountered!(4)
job 43 is complete!
job 44 is complete!
job 45 is complete!
job 46 is complete!
job 47

job 391 is complete!
job 392 is complete!
job 393 is complete!
job 394 is complete!
job 395 is complete!
job 396 is complete!
job 397 is complete!
job 398 is complete!
job 399 is complete!
job 400 is complete!
job 401 is complete!
job 402 is complete!
job 403 is complete!
job 404 is complete!
job 405 is complete!
job 406 is complete!
job 407 is complete!
job 408 is complete!
job 409 is complete!
job 410 is complete!
Bad input line encountered!(9)
job 412 is complete!
job 413 is complete!
job 414 is complete!
job 415 is complete!
job 416 is complete!
job 417 is complete!
job 418 is complete!
job 419 is complete!
job 420 is complete!
job 421 is complete!
job 422 is complete!
job 423 is complete!
job 424 is complete!
job 425 is complete!
job 426 is complete!
job 427 is complete!
job 428 is complete!
job 429 is complete!
job 430 is complete!
job 431 is complete!
job 432 is complete!
job 433 is complete!
job 434 is complete!
job 435 is complete!
job 436 is complete!
job 437 is complete!
job

# Create Postgres Queries

In [ ]:
df = pd.read_csv("ticketSearchResults-2015-02-09-20_48_16.txt", sep="~")
df = df.fillna("NULL")
df

In [ ]:
file = open('test.sql', 'w')
for i in df.index:
    file.write("INSERT INTO geocd_nj (ticket, dt, tm, hder, county, excav_comp, inter1, inter2) VALUES(%d, '%s', '%s', '%s', '%s', '%s', '%s', '%s');\n"
         % (df.loc[i, 'Ticket'],
            df.loc[i, 'Call Dt/Tm'].split(' ')[0],
            df.loc[i, 'Call Dt/Tm'].split(' ')[1],
            df.loc[i, 'Header'],
            df.loc[i, 'County'],
            df.loc[i, 'Excav Company'].replace("'", "''"),
            df.loc[i, 'Inter1'].replace("'", "''"),
            df.loc[i, 'Inter2'].replace("'", "''")))
    file.write("UPDATE geocd_nj SET rating = g.rating, lon = ST_X(g.geomout),lat = ST_Y(g.geomout), st_no = (addy).address,st_nm = (addy).streetname, city = (addy).location,state_nm = (addy).stateabbrev,zip = (addy).zip FROM geocode('%s %s, %s, NJ', 1) As g WHERE ticket = %d;\n"
         % (df.loc[i, 'Address'],
            df.loc[i, 'Street'].replace("'", "''"),
            df.loc[i, 'City'],
            df.loc[i, 'Ticket']))
file.close()